<a href="https://colab.research.google.com/github/R-Mosolov/sociology-scientometric-analysis/blob/main/parse_articles_content.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
from bs4 import BeautifulSoup
import urllib.request
import requests
import csv
import pandas as pd
import random
import time
import os

In [25]:
# '''
# Show an article content
# '''
# fp = urllib.request.urlopen("https://cyberleninka.ru/article/n/metodologicheskie-problemy-sravnitelnogo-analiza-paradigm-tehnicheskoy-realnosti")
# mybytes = fp.read()

# html_doc = mybytes.decode("utf8")
# fp.close()

# # Get HTML structure to parse
# soup = BeautifulSoup(html_doc, 'html.parser')
# article_html = soup

# article_html

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.1//EN" "http://www.w3.org/TR/xhtml11/DTD/xhtml11.dtd">

<html xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<meta content="telephone=no" name="format-detection"/>
<link href="/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
<link href="https://cyberleninka.ru/article/n/metodologicheskie-problemy-sravnitelnogo-analiza-paradigm-tehnicheskoy-realnosti" rel="canonical"/>
<title>Методологические проблемы сравнительного анализа парадигм технической реальности – тема научной статьи по социологическим наукам читайте бесплатно текст научно-исследовательской работы в электронной библиотеке КиберЛенинка</title>
<meta content="научная статья бесплатно на тему Методологические проблемы сравнительного анализа парадигм технической реальности текст научной работы по социологическим наукам из научного журнала Известия

In [26]:
# article_html.body.find('i', attrs={ 'itemprop': 'keywords' }).find_all('span')

[<span class="hl to-search" title="Найти все статьи с этим ключевым словом">ТЕХНИЧЕСКАЯ РЕАЛЬНОСТЬ</span>,
 <span class="hl to-search" title="Найти все статьи с этим ключевым словом">ПАРАДИГМА</span>,
 <span class="hl to-search" title="Найти все статьи с этим ключевым словом">ФИЛОСОФСКИЕ ОСНОВАНИЯ</span>,
 <span class="hl to-search" title="Найти все статьи с этим ключевым словом">МЕТОДОЛОГИЯ</span>,
 <span class="hl to-search" title="Найти все статьи с этим ключевым словом">КОМПАРАТИВИСТИКА</span>]

In [4]:
# '''
# Define parse targets with usefull information
# '''
# views = article_html.body.find('div', attrs={ 'class': 'views' }).get_text()
# downloads = article_html.body.find('div', attrs={ 'class': 'downloads' }).get_text()
# journal_title = article_html.body.find('div', attrs={ 'class': 'half' }).span.a.get_text()
# journal_link = article_html.body.find('div', attrs={ 'class': 'half' }).span.a['href']
# abstract = article_html.body.find('div', attrs={ 'class': 'abstract' }).p.get_text()

# # Integrate all key words
# key_words_html = article_html.body.find('i', attrs={ 'itemprop': 'keywords' }).find_all('span')
# key_words = []
# for key_word in key_words_html:
#   key_words.append(key_word.get_text().lower())

# # Integrate all article paragraphs
# article_text_html = article_html.body.find_all('p')
# article_text = []
# for article_paragraph in article_text_html:
#   article_text.append(article_paragraph.get_text())

# print('--- Содержимое статьи ---')
# print('Просмотров:', views)
# print('Скачиваний:', downloads)
# print('Название журнала:', journal_title)
# print('Гиперссылка журнала:', journal_link)
# print('Ключевые слова:', key_words)
# print('Аннотация:', abstract)
# print('Текст статьи:', article_text)

In [5]:
# '''
# Show information about the dataset with articles links
# '''
# df = pd.read_csv('/content/drive/MyDrive/Science/Datasets/cyberleninka-sociology-articles/cyberleninka-sociology-articles__1-7-4_gen-with-article-links.csv')

# print('Датасет с ссылками на статьи содержит:', len(df), 'строк')

# df.head(5)

Датасет с ссылками на статьи содержит: 49780 строк


,article_publication_date,article_author,article_title,article_license,journal_levels,article_link
0,2006,Росенко С. И.,Сравнительная характеристика социальной диффер...,Yes,ВАК,/article/n/sravnitelnaya-harakteristika-sotsia...
1,2010,Рыбаков В. Н.,Социально-педагогическиеиусловия формирования ...,Yes,ВАК,/article/n/sotsialno-pedagogicheskieiusloviya-...
2,2008,Губина Наталья Викторовна,Влияние качества жизни населения на социальный...,Yes,ВАК,/article/n/vliyanie-kachestva-zhizni-naseleniy...
3,1999,Васильев М. Е.,Информационные технологии в местном самоуправл...,No,ВАК,/article/n/informatsionnye-tehnologii-v-mestno...
4,2010,Крапивка Ирина Анатольевна,Ценностные ориентации в семейной жизни,Yes,ВАК,/article/n/tsennostnye-orientatsii-v-semeynoy-...


In [6]:
# '''
# Put all articles links into an one array
# '''
# articles_links = df['article_link']
# only_articles_links = []

# for article_link in articles_links:
#   only_articles_links.append(article_link)

In [32]:
'''
The combinated algorithm to parse articles informations
with realization of sessions rotation 
'''
# Create data structure
class Article:
  def __init__(
    self, article_link, article_views, article_downloads, journal_title,
    journal_link, key_words, abstract, article_text
  ):
    self.article_link = article_link
    self.article_views = article_views
    self.article_downloads = article_downloads
    self.journal_title = journal_title
    self.journal_link = journal_link
    self.key_words = key_words
    self.abstract = abstract
    self.article_text = article_text

# Set condition to clear a buffer stopping parser's work
isColumnName = True
EMPTY = 'EMPTY'
df = pd.read_csv('/content/drive/MyDrive/Science/Datasets/cyberleninka-sociology-articles/cyberleninka-sociology-articles__1-7-4_gen-with-article-links.csv')

# Put all articles links into an one array
articles_links = df['article_link']
only_articles_links = []

for article_link in articles_links:
  only_articles_links.append(article_link)

# Initialize proxies for parsing
proxies = pd.read_excel('/content/drive/MyDrive/Science/Datasets/proxies/proxies.xlsx')

proxies_with_ports = [
  proxies['proxy_with_port'][0],
  proxies['proxy_with_port'][1],
  proxies['proxy_with_port'][2]
]
proxy_login = proxies['login'][0]
proxy_password = proxies['password'][0]

ip_addresses = [
  'http://' + proxy_login + ':' + proxy_password + '@' + proxies_with_ports[0],
  'http://' + proxy_login + ':' + proxy_password + '@' + proxies_with_ports[1],
  'http://' + proxy_login + ':' + proxy_password + '@' + proxies_with_ports[2]
]
  
articles = []
counter = 0

# Remove old file
try:
  os.remove('/content/cyberleninka-sociology-articles__2-1_articles-content.csv')
except:
  pass

for article_link in only_articles_links:

  # Increase an article index
  counter += 1

  # Create an user's session
  url = "https://cyberleninka.ru" + str(article_link)

  user_session = requests.Session()
  proxies = {
    'http': ip_addresses[counter % 3],
    'https': ip_addresses[counter % 3]
  }

  response = user_session.get(url, proxies=proxies)
  html_doc = response.content

  # Get HTML structure to parse
  soup = BeautifulSoup(html_doc, 'html.parser')
  article_html = soup

  # Get an article views
  article_views = article_html.body.find('div', attrs={ 'class': 'views' }).get_text()
  if article_views:
    article_views = article_views
  else:
    article_views = EMPTY

  # Get an article downloads
  article_downloads = article_html.body.find('div', attrs={ 'class': 'downloads' }).get_text()
  if article_downloads:
    article_downloads = article_downloads
  else:
    article_downloads = EMPTY

  # Get a journal title
  journal_title = article_html.body.find('div', attrs={ 'class': 'half' }).span.a.get_text()
  if journal_title:
    journal_title = journal_title
  else:
    journal_title = EMPTY

  # Get a journal link
  journal_link = article_html.body.find('div', attrs={ 'class': 'half' }).span.a['href']
  if journal_link:
    journal_link = journal_link
  else:
    journal_link = EMPTY

  # Get an article key words
  try:
    key_words_html = article_html.body.find('i', attrs={ 'itemprop': 'keywords' }).find_all('span')
    key_words = []
    for key_word in key_words_html:
      key_words.append(key_word.get_text().lower())

    if key_words:
      key_words = key_words
    else:
      key_words = EMPTY
  except:
    key_words = EMPTY

  # Get an article abstract
  try:
    abstract = article_html.body.find('div', attrs={ 'class': 'abstract' }).p.get_text()
    if abstract:
      abstract = abstract
    else:
      abstract = EMPTY
  except:
    abstract = EMPTY

  # Get an article text
  try:
    article_text_html = article_html.body.find_all('p')
    article_text = []
    for article_paragraph in article_text_html:
      article_text.append(article_paragraph.get_text())

    if article_text:
      article_text = article_text
    else:
      article_text = EMPTY
  except:
    abstract = EMPTY
  
  # Put data about an article to main array
  articles.append(Article(
    article_link, article_views, article_downloads, journal_title,
    journal_link, key_words, abstract, article_text
  ))

  # Save parsed data as an Excel file
  with open('cyberleninka-sociology-articles__2-1_articles-content.csv', 'a') as csvfile:
    writer = csv.writer(csvfile)
    if isColumnName == True:
      writer.writerow([
        'article_link',
        'article_views',
        'article_downloads',
        'journal_title',
        'journal_link',
        'key_words',
        'abstract',
        'article_text'
      ])
      isColumnName = False
    for article in articles:
      writer.writerow([
        article.article_link,
        article.article_views,
        article.article_downloads,
        article.journal_title,
        article.journal_link,
        article.key_words,
        article.abstract,
        article.article_text
      ])

  # Clear buffer to minimize stoping the parser work
  articles = []

  # Set time interval between user sessions
  time_interval = random.randint(1, 1) # TODO: Change it before running the parser
  time.sleep(time_interval)
  print('Now, the following proxy have used:', ip_addresses[counter % 3].split('@')[1])
  print('Sleep finished in', time_interval, 's.')

Now, the following proxy have used: 5.101.71.237:8000
Sleep finished in 1 s.
Now, the following proxy have used: 45.81.79.24:8000
Sleep finished in 1 s.
Now, the following proxy have used: 5.101.68.214:8000
Sleep finished in 1 s.
Now, the following proxy have used: 5.101.71.237:8000
Sleep finished in 1 s.
Now, the following proxy have used: 45.81.79.24:8000
Sleep finished in 1 s.
Now, the following proxy have used: 5.101.68.214:8000
Sleep finished in 1 s.
Now, the following proxy have used: 5.101.71.237:8000
Sleep finished in 1 s.
Now, the following proxy have used: 45.81.79.24:8000
Sleep finished in 1 s.
Now, the following proxy have used: 5.101.68.214:8000
Sleep finished in 1 s.
Now, the following proxy have used: 5.101.71.237:8000
Sleep finished in 1 s.
Now, the following proxy have used: 45.81.79.24:8000
Sleep finished in 1 s.
Now, the following proxy have used: 5.101.68.214:8000
Sleep finished in 1 s.
Now, the following proxy have used: 5.101.71.237:8000
Sleep finished in 1 s.
Now

KeyboardInterrupt: ignored